In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

# Paths to the dataset
train_data_dir = "pcb_data/train"  # Replace with your training folder path
test_data_dir = "pcb_data/test"    # Replace with your test folder path

# Image dimensions
# Increased resolution for better feature extraction
IMG_HEIGHT, IMG_WIDTH = 128, 128
BATCH_SIZE = 64                   # Larger batch size for smoother gradients

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,          # Normalize pixel values
    rotation_range=30,          # Random rotation
    width_shift_range=0.3,      # Horizontal shift
    height_shift_range=0.3,     # Vertical shift
    shear_range=0.3,            # Shearing transformation
    zoom_range=0.3,             # Random zoom
    horizontal_flip=True,       # Flip images horizontally
    fill_mode="nearest"         # Fill in missing pixels
)

# Rescale only for validation/testing (no augmentation)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="binary"  # Binary classification: "good" (1) and "bad" (0)
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

# Option 1: Use a Sequential CNN model
model = Sequential([
    # Convolutional layers
    Conv2D(32, (3, 3), activation="relu",
           input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation="relu"),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(32, (3, 3), activation="relu", kernel_regularizer=l2(0.01)),
    # Add pooling layer after the last convolutional layer
    MaxPooling2D(pool_size=(2, 2)),

    Dropout(0.5),  # Add dropout here for regularization

    # Flatten the output for the fully connected layers
    Flatten(),

    # Fully connected layers
    Dense(256, activation="relu"),
    Dropout(0.5),  # Prevent overfitting
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")  # Sigmoid for binary classification
])

# Option 2: Use MobileNetV2 (pretrained feature extractor)
# Uncomment to use MobileNetV2
# base_model = MobileNetV2(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), include_top=False, weights="imagenet")
# base_model.trainable = False  # Freeze the base model
# model = Sequential([
#     base_model,
#     Flatten(),
#     Dense(128, activation="relu"),
#     Dropout(0.5),
#     Dense(1, activation="sigmoid")
# ])

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=0.0001),  # Lower learning rate for stability
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Callbacks for training
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    "best_model.h5", monitor="val_loss", save_best_only=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=30,  # Increase epochs to allow more learning
    validation_data=test_generator,
    callbacks=[early_stopping, model_checkpoint]
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Plot training and validation accuracy/loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.title("Accuracy Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Loss Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

# Define the folder to save the plots
folder = "accuracy"
os.makedirs(folder, exist_ok=True)  # Ensure the folder exists

# Get the next available number for the filename
existing_files = [f for f in os.listdir(folder) if f.startswith(
    "training_plot") and f.endswith(".png")]
next_number = 1 + max(
    [int(f.split("_")[2].split(".")[0])
     for f in existing_files if f.split("_")[2].split(".")[0].isdigit()],
    default=0
)

# Save the plot with the next number
filename = f"training_plot_1_{next_number}.png"
plt.savefig(os.path.join(folder, filename))

print(f"Plot saved as {filename} in the '{folder}' folder.")
plt.show()

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Paths to the dataset
train_data_dir = "pcb_data/train"  # Replace with your training folder path
test_data_dir = "pcb_data/test"    # Replace with your test folder path

# Image dimensions
IMG_HEIGHT, IMG_WIDTH = 128, 128  # Image size for MobileNetV2
BATCH_SIZE = 64                   # Larger batch size for smoother gradients

# Data augmentation and normalization for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,          # Normalize pixel values
    rotation_range=40,          # Increased random rotation
    width_shift_range=0.4,      # Increased horizontal shift
    height_shift_range=0.4,     # Increased vertical shift
    shear_range=0.4,            # Increased shearing transformation
    zoom_range=0.4,             # Increased random zoom
    horizontal_flip=True,       # Flip images horizontally
    fill_mode="nearest"         # Fill in missing pixels
)

# Rescale only for validation/testing (no augmentation)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

# Create data generators
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="binary"  # Binary classification: "good" (1) and "bad" (0)
)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode="binary"
)

# Use Pretrained MobileNetV2 as the feature extractor
base_model = MobileNetV2(input_shape=(
    IMG_HEIGHT, IMG_WIDTH, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Freeze the base model to use it as a feature extractor

# Build the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation="relu"),  # Fully connected layer
    # Reduced dropout to prevent over-regularization
    Dropout(0.3),
    Dense(1, activation="sigmoid")  # Sigmoid for binary classification
])

# Compile the model with a reduced learning rate
model.compile(
    # Extremely low learning rate for stability
    optimizer=Adam(learning_rate=0.000001),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Callbacks for training
early_stopping = EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint(
    "best_model.h5", monitor="val_loss", save_best_only=True)

# Train the model
history = model.fit(
    train_generator,
    epochs=30,  # Increase epochs for better optimization
    validation_data=test_generator,
    callbacks=[early_stopping, model_checkpoint]
)

# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

# Plot training and validation accuracy/loss
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history["accuracy"], label="Train Accuracy")
plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
plt.title("Accuracy Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.title("Loss Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()

# Define the folder to save the plots
folder = "accuracy"
os.makedirs(folder, exist_ok=True)  # Ensure the folder exists

# Get the next available number for the filename
existing_files = [f for f in os.listdir(folder) if f.startswith(
    "training_plot") and f.endswith(".png")]
next_number = 1 + max(
    [int(f.split("_")[2].split(".")[0])
     for f in existing_files if f.split("_")[2].split(".")[0].isdigit()],
    default=0
)

# Save the plot with the next number
filename = f"training_plot_2_{next_number}.png"
plt.savefig(os.path.join(folder, filename))

print(f"Plot saved as {filename} in the '{folder}' folder.")

plt.show()